In [13]:
import duckdb

# 使用with语句自动关闭连接
with duckdb.connect("/Users/zihao_/Documents/coding/Langchain_chatwithdata/database/central_analytics.duckdb") as con:
    # 查看数据库中的所有表
    print("数据库中的表:")
    tables = con.execute("SELECT table_name FROM file_metadata").fetchall()
    for table in tables:
        print(f"- {table[0]}")
        
    # 打印指定表的基本信息
    tables_to_analyze = [
        "价格配置_data_transposed",
        "用户分层画像_data_all_clean",
        "上险数_03_data_截止 202504_clean",
        "订单观察_聚合结果"
    ]

    print("表的基本信息:")
    for table_name in tables_to_analyze:
        # 获取表的列信息
        columns_info = con.execute(f"PRAGMA table_info('{table_name}')").fetchall()
        
        # 获取表的行数
        row_count = con.execute(f"SELECT COUNT(*) FROM '{table_name}'").fetchone()[0]
        
        # 打印表的基本信息
        print(f"\n表名: {table_name}")
        print(f"行数: {row_count}")
        print("列信息:")
        for col in columns_info:
            col_name = col[1]
            col_type = col[2]
            print(f"  - {col_name} ({col_type})")
        
        # 打印随机5行数据的预览（而不是前5行）
        print("\n数据预览 (随机5行):")
        preview = con.execute(f"SELECT * FROM '{table_name}' ORDER BY RANDOM() LIMIT 5").fetchall()
        column_names = [col[1] for col in columns_info]
        
        # 打印列名
        print("  | " + " | ".join(column_names) + " |")
        print("  |" + "---|" * len(column_names))
        
        # 打印数据
        for row in preview:
            print("  | " + " | ".join(str(val) for val in row) + " |")
        
        print("\n" + "-"*50)

数据库中的表:
- 价格配置_data_transposed
- 用户分层画像_data_all_clean
- 上险数_03_data_截止 202504_clean
- 订单观察_聚合结果
表的基本信息:

表名: 价格配置_data_transposed
行数: 469669
列信息:
  - 品牌 (VARCHAR)
  - 车型 (VARCHAR)
  - 销售状态 (VARCHAR)
  - 型号简称 (VARCHAR)
  - TP 1万1档 (VARCHAR)
  - 车身形式 (VARCHAR)
  - 燃料类型 (组) (VARCHAR)
  - 燃料类型 (VARCHAR)
  - 年 (BIGINT)
  - 月 (BIGINT)
  - Msrp (DOUBLE)
  - Tp Avg (DOUBLE)
  - 销量 (DOUBLE)

数据预览 (随机5行):
  | 品牌 | 车型 | 销售状态 | 型号简称 | TP 1万1档 | 车身形式 | 燃料类型 (组) | 燃料类型 | 年 | 月 | Msrp | Tp Avg | 销量 |
  |---|---|---|---|---|---|---|---|---|---|---|---|---|
  | 东风风光 | 风光580 | 在产在销 | 1.5T 无级 红星版 豪华型 | 10万以下 | SUV | 燃油 | Gasoline | 2021 | 12 | 99900.0 | 91400.0 | 5.212536 |
  | 荣威 | RX5 | 停产停销 | 1.5T 自动 尊荣旗舰版 | 22-23万 | SUV | 新能源 | PHEV | 2019 | 4 | 285900.0 | 223800.0 | 268.75009675 |
  | 捷途 | 捷途X70 | 在产在销 | 1.6T 双离合 悦享MAX 5座 | 12-13万 | SUV | 燃油 | Gasoline | 2023 | 12 | 137900.0 | 124500.0 | None |
  | 五十铃 | MU-X | 停产停销 | 1.9D 自动 四驱 X-POWER版 7座 | 24-25万 | SUV | 燃油 | Diesel | 2019 | 4 | 248800.0 | 24790

## 订单表

In [4]:
import duckdb
import pandas as pd
from datetime import datetime

# 连接到DuckDB数据库
conn = duckdb.connect("/Users/zihao_/Documents/coding/Langchain_chatwithdata/database/central_analytics.duckdb")

# 查询订单观察_聚合结果表，筛选5月13日至5月22日的数据
query = """
SELECT *
FROM "订单观察_聚合结果"
WHERE "当前日期" BETWEEN '2025-05-13' AND '2025-05-22'
ORDER BY "当前日期"
"""

try:
    # 执行查询
    result_df = conn.execute(query).fetchdf()
    
    # 显示结果信息
    print(f"查询成功！共获取 {len(result_df)} 条记录")
    print("\n数据预览:")
    print(result_df.head())
    
    # 保存结果到CSV文件
    output_path = "/Users/zihao_/Documents/coding/dataset/订单观察_5月13日至22日.csv"
    result_df.to_csv(output_path, index=False)
    print(f"\n数据已保存至: {output_path}")
    
    # 可选：保存为Parquet格式
    parquet_path = "/Users/zihao_/Documents/coding/dataset/订单观察_5月13日至22日.parquet"
    result_df.to_parquet(parquet_path)
    print(f"数据已保存至: {parquet_path}")
    
    # 按日期统计订单数、锁单数和交付数
    daily_stats = result_df.groupby("当前日期").agg({
        "订单数": "sum",
        "锁单数": "sum",
        "交付数": "sum"
    }).reset_index()
    
    print("\n按日期统计:")
    print(daily_stats)
    
except Exception as e:
    print(f"查询出错: {e}")
finally:
    # 关闭连接
    conn.close()

查询成功！共获取 1440 条记录

数据预览:
        当前日期     车型 车型分组             型号简称 性别  订单数  锁单数  交付数  平均年龄   平均开票价格
0 2025-05-13   智己L6  DM1       全新智己L6 Max  男  205  352    0  35.2   3039.5
1 2025-05-13   智己L6  DM1       全新智己L6 Max  女  141  258    0  32.1  14397.4
2 2025-05-13   智己L6  DM1      全新智己L6 Max+  女   78  193    0  32.5   5725.6
3 2025-05-13  智己LS7  LS7  LS7 Max Lux RWD  男    0    0    0   0.0      0.0
4 2025-05-13  智己LS6  CM1  全新LS6 超长续航灵蜥智驾版  男    0    0    0   0.0      0.0

数据已保存至: /Users/zihao_/Documents/coding/dataset/订单观察_5月13日至22日.csv
数据已保存至: /Users/zihao_/Documents/coding/dataset/订单观察_5月13日至22日.parquet

按日期统计:
        当前日期  订单数   锁单数  交付数
0 2025-05-13  758  1332   99
1 2025-05-14  154   533   78
2 2025-05-15   78   230  117
3 2025-05-16   72   153   92
4 2025-05-17  145   330   47
5 2025-05-18  155   359   34
6 2025-05-19   74   136  133
7 2025-05-20   65   124  118
8 2025-05-21   65    85  124
9 2025-05-22   60    88  106
